In [65]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [139]:
import pandas as pd
pd.set_option('max_colwidth',300)

In [66]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Step 1
- Load the train and test sets
- Check the schema, the variables have their right types?
- If not, how to correctly load the datasets?

In [67]:
### INSERT YOUR CODE HERE
train = sqlContext.read.format('com.databricks.spark.csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .option('mode', 'DROPMALFORMED')\
                .load('../data/titanic/train.csv')
train.show()
### INSERT YOUR CODE HERE
test = sqlContext.read.format('com.databricks.spark.csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .option('mode', 'DROPMALFORMED')\
                .load('../data/titanic/test.csv')
test.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [68]:
train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### Step 2
- Explore the features of your dataset
- You can use DataFrame's ***describe*** method to get summary statistics
    - hint: ***toPandas*** may be useful to ease the manipulation of small dataframes
- Are there any ***NaN*** values in your dataset?
- If so, define value/values to fill these ***NaN*** values
    - hint: ***na*** property of DataFrames provide several methods of handling NA values

In [69]:
# Calculating summary statistics and turning it into Pandas DF
train_desc = train.describe().toPandas().set_index('summary')
train_desc

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [ ]:
# Computing correlations between Survived and some features
print({col:train.stat.corr('Survived',col) for col in ['Pclass','Age','SibSp','Parch','Fare']})

# Checking which columns have NULL values
print({col:train.where(train[col].isNull()).count() for col in train.columns})

# Taking the mean age from the Pandas DF
ageMean = float(train_desc.loc['mean']['Age'])
print(ageMean)

embarkedMode = train.groupby("Embarked").count().sort("count", ascending=False).take(1)[0][0]

print(embarkedMode)

In [97]:
trainFilled = train.na.fill({"Age": ageMean, "Embarked": embarkedMode})\
.drop("Cabin")
testFilled = test.na.fill({"Age": ageMean, "Embarked": embarkedMode})\
.drop("Cabin")

print({col:trainFilled.where(trainFilled[col].isNull()).count() for col in trainFilled.columns})

{'PassengerId': 0, 'Survived': 0, 'Pclass': 0, 'Name': 0, 'Sex': 0, 'Age': 0, 'SibSp': 0, 'Parch': 0, 'Ticket': 0, 'Fare': 0, 'Embarked': 0}


### Step 3
- How to handle categorical features?
    - hint: check the Estimators and Transformers
- Assemble all desired features into a Vector using the VectorAssembler Transformer
- Make sure to end up with a DataFrame with two columns: ***Survived*** and ***vFeatures***

In [181]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.util import MLUtils

m1 = StringIndexer(inputCol="Name", outputCol="Name1").fit(trainFilled)
transformed=m1.transform(trainFilled)

m2 = StringIndexer(inputCol="Sex", outputCol="Sex1").fit(transformed)
transformed=m2.transform(transformed)

m3 = OneHotEncoder(dropLast=True, inputCol="Sex1", outputCol="Sex2")
transformed=m3.transform(transformed)

m4 = StringIndexer(inputCol="Embarked", outputCol="Embarked1", handleInvalid="skip").fit(transformed)
transformed=m4.transform(transformed)

m5 = OneHotEncoder(dropLast=False, inputCol="Embarked1", outputCol="Embarked2")
transformed=m5.transform(transformed)

m6 = StringIndexer(inputCol="Ticket", outputCol="Ticket1").fit(transformed)
transformed=m6.transform(transformed)

m7 = OneHotEncoder(dropLast=False, inputCol="Pclass", outputCol="Pclass1")
transformed=m7.transform(transformed)

transformed.select("Embarked2", "Sex2", "Ticket1", "Pclass1").show()

+-------------+-------------+-------+-------------+
|    Embarked2|         Sex2|Ticket1|      Pclass1|
+-------------+-------------+-------+-------------+
|(3,[0],[1.0])|(1,[0],[1.0])|  257.0|(4,[3],[1.0])|
|(3,[1],[1.0])|    (1,[],[])|  608.0|(4,[1],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|  292.0|(4,[3],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|   46.0|(4,[1],[1.0])|
|(3,[0],[1.0])|(1,[0],[1.0])|  425.0|(4,[3],[1.0])|
|(3,[2],[1.0])|(1,[0],[1.0])|  269.0|(4,[3],[1.0])|
|(3,[0],[1.0])|(1,[0],[1.0])|  438.0|(4,[1],[1.0])|
|(3,[0],[1.0])|(1,[0],[1.0])|   12.0|(4,[3],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|   27.0|(4,[3],[1.0])|
|(3,[1],[1.0])|    (1,[],[])|   55.0|(4,[2],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|   65.0|(4,[3],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|  510.0|(4,[1],[1.0])|
|(3,[0],[1.0])|(1,[0],[1.0])|  658.0|(4,[3],[1.0])|
|(3,[0],[1.0])|(1,[0],[1.0])|    0.0|(4,[3],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|  635.0|(4,[3],[1.0])|
|(3,[0],[1.0])|    (1,[],[])|  336.0|(4,[2],[1.0])|
|(3,[2],[1.0

In [184]:
assembler = VectorAssembler(inputCols = ["Pclass1","Sex2","Age","SibSp",
                                         "Parch","Fare","Embarked2"
                                        ], outputCol = "vfeatures")

dfVec = assembler.transform(transformed)

In [185]:
assembled = dfVec.select("Survived","vfeatures")

In [186]:
assembled.limit(5).toPandas()

,Survived,vfeatures
0,0,"(0.0, 0.0, 0.0, 1.0, 1.0, 22.0, 1.0, 0.0, 7.25, 1.0, 0.0, 0.0)"
1,1,"(0.0, 1.0, 0.0, 0.0, 0.0, 38.0, 1.0, 0.0, 71.2833, 0.0, 1.0, 0.0)"
2,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 26.0, 0.0, 0.0, 7.925, 1.0, 0.0, 0.0)"
3,1,"(0.0, 1.0, 0.0, 0.0, 0.0, 35.0, 1.0, 0.0, 53.1, 1.0, 0.0, 0.0)"
4,0,"(0.0, 0.0, 0.0, 1.0, 1.0, 35.0, 0.0, 0.0, 8.05, 1.0, 0.0, 0.0)"


In [190]:
#Same as above steps. compact

from pyspark.ml import Pipeline

# only string indexing binary columns
# 

binary = ['Sex','Embarked']
categorical = ['Pclass', 'SibSp', 'Embarked'] 
categorical = [c + '_index' if c in binary else c for c in categorical]

indexers = [StringIndexer(inputCol=col, outputCol=col +"_index") for col in binary]
encoder_indexers = indexers + [OneHotEncoder(inputCol=column, outputCol=column+"_encoded") for column in categorical] 
encoder_pipeline = Pipeline(stages = encoder_indexers)

compact_encoded = encoder_pipeline.fit(trainFilled).transform(trainFilled)
compact_encoded.limit(3).toPandas()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Sex_index,Embarked_index,Pclass_encoded,SibSp_encoded,Embarked_index_encoded
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0.0,0.0,"(0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C,1.0,1.0,"(0.0, 1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0)"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1.0,0.0,"(0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)"


### Step 4
- Apply a normalization Estimator of your choice to the ***features*** vector obtained in Step 3

In [194]:
from pyspark.ml.feature import StandardScaler

scalar2 = StandardScaler().setInputCol("vfeatures").setOutputCol("scaledFeat").setWithStd(True).setWithMean(True)
scalar2Model = scalar2.fit(assembled)
scalar2Normalized = scalar2Model.transform(assembled)

scalar2Normalized.limit(2).toPandas()

,Survived,vfeatures,scaledFeat
0,0,"(0.0, 0.0, 0.0, 1.0, 1.0, 22.0, 1.0, 0.0, 7.25, 1.0, 0.0, 0.0)","[0.0, -0.565367891708, -0.509865182364, 0.902080720168, 0.73728104523, -0.592148025766, 0.432550428042, -0.473407724568, -0.502163136516, 0.61549273983, -0.481772097386, -0.307389700253]"
1,1,"(0.0, 1.0, 0.0, 0.0, 0.0, 38.0, 1.0, 0.0, 71.2833, 0.0, 1.0, 0.0)","[0.0, 1.76677466159, -0.509865182364, -1.10730408401, -1.35481262133, 0.638430443947, 0.432550428042, -0.473407724568, 0.786403617835, -1.62289106094, 2.07334063339, -0.307389700253]"


### Step 5
- Instead of doing transformations on separate steps, put everything together with a Pipeline

In [ ]:
from pyspark.ml.pipeline import Pipeline

### INSERT YOUR CODE HERE

### Step 6
- Train a classifier of your choice (for instance, Random Forest) using your dataset of LabeledPoints
- Make predictions for the training data
- Use the evaluators to find the Area Under ROC and Accuracy of your model
- How is your model performing? Try to tune its parameters

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

### INSERT YOUR CODE HERE

### Step 7
- Take a look at the test data - use DataFrame's ***createOrReplaceTempView*** method to perform SQL queries over the data
    - hint: check if there are any NULL values in the dataset - if so, handle them
- Apply the transformations to the test data
    - hint: include the model to the pipeline
- Make predictions using the model previously trained and the transformed test data

In [ ]:
### INSERT YOUR CODE HERE

### Step 8

- Load the answers for the ***test*** data
- Combine it with your predictions into a single DataFrame
- Use the evaluator you created on ***Step 6***
- What was your score?

In [ ]:
### INSERT YOUR CODE HERE

In [64]:
sc.stop()